## Import the packages

In [1]:
import random
import operator

## Random cost matrix from each city to all other cities

In [2]:
def SetCostMatrix(num):
    cmatrix = {}
    for i in range(1, num + 1):
        for j in range(1, num + 1):
            if i == j:
                cmatrix[(i, j)] = 0
            else:
                cmatrix[(i, j)] = random.randint(10, 100)
    return cmatrix

In [3]:
num_cities = SetCostMatrix(10)

## Get the cost or distance value from one city to another city

In [4]:
def GetCostVal(row, col):
    if col == 0:
        col = 1
    return num_cities[(row, col)]

## Get the possible source city and rest all target cities

In [5]:
def GetPossibilites(source, cities_count):
    cities = list(range(1, cities_count + 1))
    possibilities = {}
    for city in cities:
        dtars = cities[:]
        dtars.remove(city)
        possibilities[city] = dtars
    return possibilities

### 10 cities, therefore 10 possibilities

In [6]:
GetPossibilites(1, 10)

{1: [2, 3, 4, 5, 6, 7, 8, 9, 10],
 2: [1, 3, 4, 5, 6, 7, 8, 9, 10],
 3: [1, 2, 4, 5, 6, 7, 8, 9, 10],
 4: [1, 2, 3, 5, 6, 7, 8, 9, 10],
 5: [1, 2, 3, 4, 6, 7, 8, 9, 10],
 6: [1, 2, 3, 4, 5, 7, 8, 9, 10],
 7: [1, 2, 3, 4, 5, 6, 8, 9, 10],
 8: [1, 2, 3, 4, 5, 6, 7, 9, 10],
 9: [1, 2, 3, 4, 5, 6, 7, 8, 10],
 10: [1, 2, 3, 4, 5, 6, 7, 8, 9]}

## Recursive function to get the minimum distance from source to target cities

In [7]:
def TSPGetMinDistance(source, cities):
    if len(cities) == 1:
        return GetCostVal(source, cities[0]) + GetCostVal(cities[0], 0)
    else:
        Dist = []
        for city in cities:
            dcities = cities[:]
            dcities.remove(city)
            Dist.append(GetCostVal(source, city) + TSPGetMinDistance(city, dcities))
        return min(Dist)

In [8]:
distances = {}
possibilities = GetPossibilites(1, 10)
for source, cities in possibilities.items():
    distances[source] = TSPGetMinDistance(source, cities)

## Possible minimum distance for each source

In [9]:
distances

{1: 288,
 2: 271,
 3: 247,
 4: 250,
 5: 275,
 6: 235,
 7: 268,
 8: 260,
 9: 262,
 10: 264}

## The least of the least, best source

In [10]:
source_minimal = min(distances.items(), key=operator.itemgetter(1))[0]
source_minimal

6